# Related issue: https://github.com/open-mmlab/mmocr/issues/352

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-db46c46c-e2ff-fa50-9e0c-a98e4d2916da)


## Install libraries

In [2]:
%%capture
# Install torch dependencies: (use cu101 since colab has CUDA 10.1)
!pip install -U torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.5.0/index.html

# Install mmdetection
!pip install mmdet==2.11.0

# Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
#Install from commit https://github.com/open-mmlab/mmocr/commit/6d97d8286e333dd8aa613a59c467d6ed85d3d6a3 (i.e. with MMOCR v0.2.0)
!git checkout 6d97d8286e333dd8aa613a59c467d6ed85d3d6a3 
!pip install -r requirements.txt
!pip install -v -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

In [3]:
!nvcc -V
!gcc --version

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

1.5.0+cu101 True
2.11.0
1.3.9
10.1
GCC 7.3
0.2.0


## Run training

In [4]:
!wget https://raw.githubusercontent.com/antoniolanza1996/miscellaneous/d12b519a089cb4b8a177ca8acf0906ea20d0335a/mmocr/issues/352/dbnet_custom_config.py -O configs/textdet/dbnet/dbnet_custom_config.py

--2021-07-21 13:55:40--  https://raw.githubusercontent.com/antoniolanza1996/miscellaneous/d12b519a089cb4b8a177ca8acf0906ea20d0335a/mmocr/issues/352/dbnet_custom_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3258 (3.2K) [text/plain]
Saving to: ‘configs/textdet/dbnet/dbnet_custom_config.py’

configs/textdet/dbn 100%[===================>]   3.18K  --.-KB/s    in 0s      

2021-07-21 13:55:40 (44.8 MB/s) - ‘configs/textdet/dbnet/dbnet_custom_config.py’ saved [3258/3258]



In [5]:
%%shell

WORK_DIR="../textdet_output"
mkdir -p $WORK_DIR

python3 tools/train.py ./configs/textdet/dbnet/dbnet_custom_config.py \
  --seed 42 \
  --work-dir=$WORK_DIR \
  --cfg-options \
    load_from="https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_r50dcnv2_fpnc_sbn_1200e_icdar2015_20210325-91cef9af.pth" \
    log_config.interval=1 \
    checkpoint_config.interval=1

2021-07-21 13:55:50,834 - mmocr - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.11 (default, Jul  3 2021, 18:01:19) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.5.0+cu101
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75

CalledProcessError: ignored